<a href="https://colab.research.google.com/github/kartik703/Resume-Matcher-and-Interview-Question-Generator/blob/main/CV_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Run the app with ngrok
from pyngrok import ngrok
public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)
app.run()

Public URL: https://389f-34-48-84-94.ngrok-free.app


NameError: name 'app' is not defined

In [5]:
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2spSTpuhUshKKVq7XVFQbeG2ejK_5pFx4CNCK9tnex9ZSER5y")

# Start ngrok tunnel
public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)

Public URL: https://2f63-34-48-84-94.ngrok-free.app


In [9]:
# Install dependencies
!pip install langchain faiss-cpu transformers sentence-transformers PyPDF2 pyngrok Flask

# Import libraries
import os
from flask import Flask, request, jsonify, render_template
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import base64
from pyngrok import ngrok

# Create the templates folder and index.html file
os.makedirs("templates", exist_ok=True)
index_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Resume Matcher</title>
</head>
<body>
    <h1>Resume Matcher and Interview Question Generator</h1>
    <form id="resumeForm">
        <label for="job_description">Job Description:</label><br>
        <textarea id="job_description" name="job_description" rows="4" cols="50"></textarea><br><br>
        <label for="resume_file">Upload Resume (PDF):</label><br>
        <input type="file" id="resume_file" name="resume_file" accept=".pdf"><br><br>
        <button type="submit">Process</button>
    </form>
    <h2>Results</h2>
    <div id="results"></div>

    <script>
        document.getElementById("resumeForm").addEventListener("submit", async (e) => {
            e.preventDefault();
            const jobDescription = document.getElementById("job_description").value;
            const resumeFile = document.getElementById("resume_file").files[0];

            const reader = new FileReader();
            reader.onload = async () => {
                const base64Resume = reader.result.split(",")[1];
                const response = await fetch("/process", {
                    method: "POST",
                    headers: {
                        "Content-Type": "application/json",
                    },
                    body: JSON.stringify({
                        job_description: jobDescription,
                        resume_file: base64Resume,
                    }),
                });
                const data = await response.json();
                document.getElementById("results").innerHTML = `
                    <h3>Match Summary:</h3>
                    <p>${data.match_summary}</p>
                    <h3>Interview Questions:</h3>
                    <p>${data.interview_questions}</p>
                `;
            };
            reader.readAsDataURL(resumeFile);
        });
    </script>
</body>
</html>
"""
with open("templates/index.html", "w") as f:
    f.write(index_html_content)

# Initialize Flask app
app = Flask(__name__)

# Load Hugging Face LLM
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
llm_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Load embeddings model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

def parse_resume(file_path):
    """Extract text from a resume (PDF only)"""
    if not os.path.exists(file_path):
        raise ValueError(f"File path {file_path} does not exist.")
    if not file_path.lower().endswith(".pdf"):
        raise ValueError("Only PDF files are supported.")
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""]
    )
    return [doc.page_content for doc in text_splitter.split_documents(docs)]

def create_vector_store(resumes):
    """Create FAISS vector store from extracted resume text"""
    return FAISS.from_texts(resumes, embeddings)

def job_match(job_description, vector_store):
    """Find best matching resumes for a given job description"""
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})  # Get top 3 matches
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
    # Use a more detailed prompt
    prompt = f"""Summarize the candidate's qualifications and experience based on the following job description:
    Job Description: {job_description}
    Focus on:
    - Technical skills (e.g., Python, PyTorch, TensorFlow)
    - Projects (e.g., AI-driven solutions, NLP, deep learning)
    - Work experience (e.g., software engineering roles, AI research)
    - Education (e.g., degrees, certifications)
    Provide specific examples from the resume."""
    result = qa.invoke({"query": prompt})
    return result["result"], result["source_documents"]

def generate_interview_questions(candidate_resume):
    """Generate interview questions based on a candidate's resume"""
    # Split the resume text into chunks of 500 tokens
    max_tokens = 500
    chunks = [candidate_resume[i:i + max_tokens] for i in range(0, len(candidate_resume), max_tokens)]

    questions = []
    for chunk in chunks:
        prompt = f"""Generate 2-3 professional interview questions based on the following resume content:
        {chunk}
        Focus on:
        - Technical skills (e.g., Python, PyTorch, TensorFlow)
        - Projects (e.g., AI-driven solutions, NLP, deep learning)
        - Work experience (e.g., software engineering roles, AI research)
        - Problem-solving and collaboration skills"""
        response = llm_pipeline(prompt, max_length=200, truncation=True)
        questions.extend(response[0]['generated_text'].split("\n"))

    # Remove duplicate questions
    unique_questions = list(set(questions))
    return "\n".join(unique_questions[:5])  # Return top 5 questions

@app.route("/", methods=["GET"])
def home():
    return render_template("index.html")

@app.route("/process", methods=["POST"])
def process():
    try:
        data = request.json
        if not data:
            return jsonify({"error": "No data provided"}), 400

        job_desc = data.get("job_description")
        resume_file = data.get("resume_file")

        if not job_desc or not resume_file:
            return jsonify({"error": "Missing job_description or resume_file"}), 400

        # Save the uploaded file
        with open("temp_resume.pdf", "wb") as f:
            f.write(base64.b64decode(resume_file.encode("utf-8")))

        # Process the resume
        resume_chunks = parse_resume("temp_resume.pdf")
        vector_store = create_vector_store(resume_chunks)
        match_summary, source_docs = job_match(job_desc, vector_store)
        full_resume_text = "\n".join([doc.page_content for doc in source_docs])
        interview_questions = generate_interview_questions(full_resume_text)

        return jsonify({
            "match_summary": match_summary,
            "interview_questions": interview_questions
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run the app with ngrok
public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)
app.run()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Public URL: https://5e03-34-48-84-94.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 06:56:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 06:57:02] "POST /process HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 06:58:10] "POST /process HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 06:58:12] "POST /process HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 06:58:12] "POST /process HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 06:58:24] "POST /process HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 06:58:25] "POST /process HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 06:58:26] "POST /process HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 06:58:26] "POST /process HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 06:58:26] "POST /process 